In [ ]:
!pip install openai langchain 

In [2]:
import os
os.environ['OPENAI_API_KEY']=''

In [4]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("You Human!")

history.add_ai_message("I am more Human?")

#which exposes convienence methods for saving 
#Human messages, AI messages, and then fetching them all

In [12]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")

In [13]:
memory.save_context({"input": "hello Genius"}, 
                    {"ouput": "I am Genius.."})

In [14]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi!', additional_kwargs={}),
  AIMessage(content='whats up?', additional_kwargs={}),
  HumanMessage(content='hello Genius', additional_kwargs={}),
  AIMessage(content='I am Genius..', additional_kwargs={})]}

In [15]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

#Adding memory to chain
llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

### Conversation Buffer memory seems to be more versatile
### Conversation Buffer memory restricts the memory to window size

Both are help in just storage of memory

In [16]:
conversation.predict(input='How are you?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: How are you?
AI:

> Finished chain.


" I'm doing great! I'm feeling very productive today. I've been working on a few projects and I'm excited to see the results. How about you?"

In [17]:
conversation.predict(input='Remember, my name is King Kong')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How are you?
AI:  I'm doing great! I'm feeling very productive today. I've been working on a few projects and I'm excited to see the results. How about you?
Human: Remember, my name is King Kong
AI:

> Finished chain.


" Hi King Kong! It's nice to meet you. I'm glad to know your name."

In [ ]:
conversation.predict(input='What did I tell you first?')

In [18]:
conversation.memory.load_memory_variables({})

{'history': "Human: How are you?\nAI:  I'm doing great! I'm feeling very productive today. I've been working on a few projects and I'm excited to see the results. How about you?\nHuman: Remember, my name is King Kong\nAI:  Hi King Kong! It's nice to meet you. I'm glad to know your name."}

In [19]:
from langchain.llms import OpenAI
from langchain.memory import ConversationEntityMemory

In [20]:
#memories have their own llm!!!
memory = ConversationEntityMemory(llm=llm)

_input = {"input": "Nona & Kie are working on a Clay house"}

memory.load_memory_variables(_input)

memory.save_context(
    _input,
    {"ouput": " Play with clay is fun, right?"}
)

In [21]:
memory.load_memory_variables({"input":"who is Kie?"})

{'history': 'Human: Nona & Kie are working on a Clay house\nAI:  Play with clay is fun, right?',
 'entities': {'Kie': 'Kie is working on a Clay house with Nona.'}}

In [22]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from pydantic import BaseModel
from typing import List, Dict, Any

In [23]:
#Extracting entities using special prompt
#Note there is llm for the chain and seperately for memory
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [25]:
conversation.predict(input="Kie and Nona are working with clay")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like a fun activity! What kind of clay are they using?'

In [26]:
conversation.memory.store

{'Kie': 'Kie is working with clay.', 'Nona': 'Nona is working with clay.'}

In [27]:
conversation.prompt

PromptTemplate(input_variables=['entities', 'history', 'input'], output_parser=None, partial_variables={}, template='You are an assistant to a human, powered by a large language model trained by OpenAI.\n\nYou are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nYou are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to gen